In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
DEBUG = False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
from disentangle.analysis.plot_utils import clean_ax
from disentangle.core.tiff_reader import load_tiff
from disentangle.config_utils import load_config, get_configdir_from_saved_predictionfile
from disentangle.core.data_split_type import DataSplitType
from disentangle.scripts.evaluate import get_highsnr_data
import ml_collections

In [ ]:
        # '/home/ashesh.ashesh/training/disentangle/2402/D16-M3-S0-L0/128',
        # '/home/ashesh.ashesh/training/disentangle/2402/D16-M3-S0-L0/144',
# 2402/D16-M3-S0-L0/144
        # '/home/ashesh.ashesh/training/disentangle/2402/D16-M3-S0-L0/145'

        # '/home/ashesh.ashesh/training/disentangle/2402/D16-M3-S0-L0/165',
        # '/home/ashesh.ashesh/training/disentangle/2402/D16-M3-S0-L0/164',
        # '/home/ashesh.ashesh/training/disentangle/2402/D16-M3-S0-L0/169',

noise_levels = [5100]
pred_dir = '/group/jug/ashesh/data/paper_stats/'

usplit_fname = {5100: 'Test_P64_G32_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_165.tif',
                # 6675: 'Test_P64_G32_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_164.tif',
                # 6675: 'Test_P64_G32_M5_Sk44/pred_disentangle_colorfuljug_2403_D16-M3-S0-L0_1.tif',
                4725: 'Test_P64_G32_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_169.tif',
                228: 'Test_PNone_G32_M10_Sk0/pred_disentangle_2403_D23-M3-S0-L0_0.tif',
                
                }

denoiSplitNM_fname = {
                5100: 'Test_P128_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_128.tif',
                # 6675: 'Test_P128_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_144.tif', 
                # 6675: 'Test_P128_G64_M50_Sk44/pred_disentangle_2403_D16-M3-S0-L0_25.tif',
                4725: 'Test_P128_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_145.tif',
                228: 'Test_P128_G32_M10_Sk32/pred_disentangle_2402_D3-M3-S0-L0_32.tif',
                      }
hdn_usplit = {} #{4450: 'pred_disentangle_2402_D23-M3-S0-L0_34.tif'}

In [ ]:
def _noise_model_path(nmodel_dir):
    histfpath = None
    gmmfpath = None
    for fname in os.listdir(nmodel_dir):
        if fname.startswith('HistNoiseModel'):
            histfpath = os.path.join(nmodel_dir,fname)
        elif fname.startswith('GMMNoiseModel'):
            gmmfpath = os.path.join(nmodel_dir,fname)
    return {'gmm':gmmfpath, 'hist':histfpath}

def noise_model_paths(pred_file_name):
    """
    denoiSplitNM_fname[noise_levels[0]]
    """
    cfg = load_config(get_configdir_from_saved_predictionfile(pred_file_name))
    nmodel1_fpath_dict = None
    nmodel2_fpath_dict = None
    if 'noise_model_ch1_fpath' in cfg.model and cfg.model.noise_model_ch1_fpath is not None:
        nmodel1_fpath_dict = _noise_model_path(os.path.dirname(cfg.model.noise_model_ch1_fpath))
    if 'noise_model_ch2_fpath' in cfg.model and cfg.model.noise_model_ch2_fpath is not None:
        nmodel2_fpath_dict = _noise_model_path(os.path.dirname(cfg.model.noise_model_ch2_fpath))
    return nmodel1_fpath_dict, nmodel2_fpath_dict

def _get_noise_model(nmodel_fpath_dict):
    from disentangle.nets.gmm_noise_model import GaussianMixtureNoiseModel
    from disentangle.nets.hist_noise_model import HistNoiseModel
    nmodel_params = np.load(nmodel_fpath_dict['gmm'])
    gmm_model1 = GaussianMixtureNoiseModel(params=nmodel_params)
    
    histdata = np.load(nmodel_fpath_dict['hist'])
    hist_model = HistNoiseModel(histdata)
    return {'gmm':gmm_model1, 'hist':hist_model}

def get_noise_models(pred_file_name):
    nmodel1_fpath_dict, nmodel2_fpath_dict = noise_model_paths(pred_file_name)
    nmodel1 = _get_noise_model(nmodel1_fpath_dict)
    nmodel2 = _get_noise_model(nmodel2_fpath_dict)
    return nmodel1, nmodel2

nmodel1, nmodel2 = get_noise_models(denoiSplitNM_fname[noise_levels[0]])

In [ ]:
import torch
import numpy as np
from disentangle.analysis.plot_utils import add_subplot_axes

def get_signal_from_index(signalBinIndex, n_bin, min_signal, max_signal, histBinSize):
    querySignal_numpy = (signalBinIndex / float(n_bin) * (max_signal - min_signal) + min_signal)
    querySignal_numpy += histBinSize / 2
    querySignal_torch = torch.from_numpy(np.array(querySignal_numpy)).float()
    return querySignal_torch

def get_scaled_pdf(pdf, axymax, axymin, yval, factor=0.2):
    scaled_pdf = pdf/pdf.max()
    scaled_pdf = scaled_pdf - scaled_pdf.min()
    scaled_pdf = scaled_pdf * (axymax - axymin)*factor + yval
    return scaled_pdf


# def add_signal_value(ax, signal)

def plot_noise_model(signal1_index, signal2_index, histogramNoiseModel, gaussianMixtureNoiseModel, device, ax, linetxt_offset = 0.1):
    """Plots probability distribution P(x|s) for a certain ground truth signal.
       Predictions from both Histogram and GMM-based Noise models are displayed for comparison.
        Parameters
        ----------
        signalBinIndex: int
            index of signal bin. Values go from 0 to number of bins (`n_bin`).
        histogramNoiseModel: Histogram based noise model
        gaussianMixtureNoiseModel: GaussianMixtureNoiseModel
            Object containing trained parameters.
        device: GPU device
        """
    max_signal = histogramNoiseModel.maxv.item()
    min_signal = histogramNoiseModel.minv.item()
    n_bin = int(histogramNoiseModel.bins.item())

    histBinSize = (max_signal - min_signal) / n_bin
    signal1 = get_signal_from_index(signal1_index, n_bin, min_signal, max_signal, histBinSize).to(device)
    signal2 = get_signal_from_index(signal2_index, n_bin, min_signal, max_signal, histBinSize).to(device)

    queryObservations_numpy = np.arange(min_signal, max_signal, histBinSize)
    queryObservations_numpy += histBinSize / 2
    queryObservations = torch.from_numpy(queryObservations_numpy).float().to(device)
    
    gmm_pdf1 = gaussianMixtureNoiseModel.likelihood(queryObservations, signal1)
    gmm_pdf1 = gmm_pdf1.detach().cpu().numpy()

    gmm_pdf2 = gaussianMixtureNoiseModel.likelihood(queryObservations, signal2)
    gmm_pdf2 = gmm_pdf2.detach().cpu().numpy()

    # plt.figure(figsize=(12, 5))

    # plt.subplot(1, 2, 1)
    # plt.xlabel('Observation Bin')
    # plt.ylabel('Signal Bin')
    histogram = histogramNoiseModel.fullHist.cpu().numpy()
    ax.imshow(histogram**0.25, cmap='gray', aspect='auto')
    yval1 = signal1_index + 0.5
    yval2 = signal2_index + 0.5
    ax.axhline(y=yval1, linewidth=1, color='green', linestyle='--', alpha=0.5, label=f'{signal1.cpu().numpy():.1f}')
    ax.axhline(y=yval2, linewidth=1, color='green', linestyle='--', alpha=0.5, label=f'{signal2.cpu().numpy():.1f}')

    # plt.subplot(1, 2, 2)
    hist_pdf1 = histogramNoiseModel.likelihood(queryObservations, signal1).cpu().numpy()
    hist_pdf2 = histogramNoiseModel.likelihood(queryObservations, signal2).cpu().numpy()
    ymin, ymax = ax.get_ylim()

    pdf1 = get_scaled_pdf(gmm_pdf1, ymax, ymin, yval1)
    pdf2 = get_scaled_pdf(gmm_pdf2, ymax, ymin, yval2)

    step = histogram.shape[1]/pdf1.shape[0]
    x = np.arange(0, histogram.shape[1], step=step)
    ax.plot(x, pdf1, color='green')
    ax.plot(x, pdf2, color='green')
    
    ymin, ymax = ax.get_ylim()
    print(ymin, ymax)
    props = dict(alpha=0)
    fact1 = (signal1_index - ymin)/(ymax - ymin) + linetxt_offset
    ax.text(0.77, fact1, f'{signal1.cpu().numpy():.0f}', transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props, color='green')
    fact2 = (signal2_index - ymin)/(ymax - ymin) + linetxt_offset
    ax.text(0.02, fact2, f'{signal2.cpu().numpy():.0f}', transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props, color='green')

    # ax.legend(frameon=False, labelcolor='white', loc='upper right')


In [ ]:
_, ax = plt.subplots(figsize=(6,3))
plot_noise_model(25, 75, nmodel1['hist'], nmodel1['gmm'], 'cpu', ax, linetxt_offset=0.2)

In [ ]:
# from disentangle.utils import plotProbabilityDistribution
# signalBinIndex=60
# data_dict = plotProbabilityDistribution(signalBinIndex=signalBinIndex, 
#                             histogramNoiseModel=nmodel2['hist'],
#                             gaussianMixtureNoiseModel=nmodel2['gmm'],
#                             device='cpu')


In [ ]:
def sanity_check_config():
    data_dicts = [usplit_fname, denoiSplitNM_fname]
    for ith_data, ddict in enumerate(data_dicts):
        for noise,fname in ddict.items():
            configdir = get_configdir_from_saved_predictionfile(fname)
            config = load_config(configdir)
            assert 'synthetic_gaussian_scale' in config.data
            assert config.data.synthetic_gaussian_scale == noise, f'{ith_data} {fname}: noise: {noise}, config: {config.data.synthetic_gaussian_scale}'
    


In [ ]:
sanity_check_config()

### Loading target

In [ ]:
configdir  = get_configdir_from_saved_predictionfile(denoiSplitNM_fname[noise_levels[0]])
config = ml_collections.ConfigDict(load_config(configdir))
highsnr_data = get_highsnr_data(config, config.datadir, DataSplitType.Test)

### Loading predictions

In [ ]:
import json
usplit_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in usplit_fname.items()}
denoiSplitNM_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in denoiSplitNM_fname.items()}
hdn_usplit_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in hdn_usplit.items()}

# Undoing the offset.
for k,v in usplit_fname.items():
     with open(os.path.join(pred_dir, v.replace('.tif', '.json')),'rb') as f:
        offset = float(json.load(f)['offset'])
        usplit_data[k] = usplit_data[k] + offset

for k,v in denoiSplitNM_fname.items():
    with open(os.path.join(pred_dir, v.replace('.tif', '.json')),'rb') as f:
        offset = float(json.load(f)['offset'])
        denoiSplitNM_data[k] = denoiSplitNM_data[k] + offset


### Cropping the target to get to the same shape as the predictions

In [ ]:
shape = usplit_data[noise_levels[0]].shape
highsnr_data = highsnr_data[:, :shape[1], :shape[2]].copy()

In [ ]:
shape

In [ ]:
def sanity_check_data():
    # all shapes should be same
    for noise_level in noise_levels:
        shape = usplit_data[noise_level].shape
        if noise_level in denoiSplitNM_data:
            assert shape == denoiSplitNM_data[noise_level].shape
        if noise_level in hdn_usplit_data:
            assert shape == hdn_usplit_data[noise_level].shape
        assert shape == highsnr_data.shape
            

In [ ]:
# usplit_data[noise_levels[0]] = usplit_data[noise_levels[0]][...,::-1].copy()

In [ ]:
sanity_check_data()

In [ ]:
paper_figures_dir = '/group/jug/ashesh/data/paper_figures'
def get_output_fpath(noise_level):
    if 'ch1_fname' in config.data:
        ch1str = config.data.ch1_fname.split('.')[0].replace('/','').replace('GT_all', '')
        ch2str = config.data.ch2_fname.split('.')[0].replace('/','').replace('GT_all', '')
    else:
        ch1str = config.data.channel_1
        ch2str = config.data.channel_2
    modelid = config.workdir.strip('/').split('/')[-1]

    output_filepath =os.path.join(paper_figures_dir, f'{modelid}_{noise_level}_{ch1str}_{ch2str}.png')
    output_filepath
    return output_filepath

In [ ]:
import numpy as np
def get_noisy_data(noise_level):
    return highsnr_data + np.random.normal(0, noise_level, highsnr_data.shape)

In [ ]:
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import add_pixel_kde, clean_ax
from disentangle.core.psnr import RangeInvariantPsnr
import seaborn as sns   

#### inset specific
inset_rect=[0.05, 0.05, 0.4, 0.2]
inset_min_labelsize=10
color_ch_list=['goldenrod', 'cyan']
color_pred='red'
# insetplot_xmax_value = 30000
# insetplot_xmin_value = -8000
# paviaatn 
insetplot_xmax_value = 1500
insetplot_xmin_value = -100

plt_dsample = 4
####
data_idx = 0
img_sz = 3
ncol_imgs = 5
nrow_imgs = 2
example_spacing = 1
grid_factor = 5
nimgs = 1
noise_level = noise_levels[0]
# extra spacing for c0. It does not work. Don't know why. I think there is some integer division happening.
c0_extra = 1

noisy_data = get_noisy_data(noise_level)

# for subscripts and superscripts
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)

def get_psnr_str(prediction, ch_idx):
    return f'{RangeInvariantPsnr(highsnr_data[data_idx,...,ch_idx][None], prediction[data_idx,...,ch_idx][None]).item():.1f}' 

def add_psnr_str(ax_, psnr):
    """
    Add psnr string to the axes
    """
    textstr = f'PSNR\n{psnr}'
    props = dict(
        boxstyle='round', 
        facecolor='gray', alpha=0.3)
    # place a text box in upper left in axes coords
    ax_.text(0.05, 0.95, textstr, transform=ax_.transAxes, fontsize=11,
            verticalalignment='top', bbox=props, color='white')

# extra spacing for the first and the last column.
fig_w = ncol_imgs * img_sz + 2*c0_extra/grid_factor
fig_h = int(img_sz * nrow_imgs + (example_spacing * (nimgs - 1)) / grid_factor )
fig = plt.figure(figsize=(fig_w, fig_h))
gs = GridSpec(nrows=int(grid_factor * fig_h), ncols=int(grid_factor * fig_w), hspace=0.2, wspace=0.2)
grid_img_sz = img_sz * grid_factor

# input
ax_temp = fig.add_subplot(gs[:grid_img_sz,:grid_img_sz])
ax_temp.imshow(np.mean(noisy_data[data_idx], axis=-1), cmap='magma')
legend_ax = ax_temp

clean_ax(ax_temp)

# ax[0,0].set_title('Input')
ax_temp = fig.add_subplot(gs[:grid_img_sz, (c0_extra+grid_img_sz):(c0_extra + grid_img_sz * 2)])
ax_temp.imshow(noisy_data[data_idx,:,:,0], cmap='magma')
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [noisy_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,0]],
                                  inset_min_labelsize,
                                  label_list=['NoisyCh1','Ch1'],
                                  plot_kwargs_list=[{'linestyle':'--'}, {}],
                                  color_list=[color_ch_list[0],color_ch_list[0]],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2, c0_extra+grid_img_sz:c0_extra + grid_img_sz * 2])
ax_temp.imshow(noisy_data[data_idx,:,:,1], cmap='magma')
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [noisy_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,1]],
                                  inset_min_labelsize,
                                  label_list=['NoisyCh2','Ch2'],
                                  color_list=[color_ch_list[1],color_ch_list[1]],
                                  plot_kwargs_list=[{'linestyle':'--'},{}],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[:grid_img_sz, c0_extra+grid_img_sz * 2:c0_extra+grid_img_sz * 3])
ax_temp.imshow(usplit_data[noise_level][data_idx,...,0], cmap='magma')
# inset_ax = add_pixel_kde(ax_temp,
#                                   inset_rect,
#                                   [highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
#                                    noisy_data[data_idx,::plt_dsample,::plt_dsample,0],
#                                   usplit_data[noise_level][data_idx,::plt_dsample,::plt_dsample,0]],
#                                   inset_min_labelsize,
#                                   label_list=['Ch1','input', 'Pred1'],
#                                   color_list=[color_ch_list[0],color_ch_list[0], color_pred],
#                                   plot_kwargs_list=[{},{'linestyle':'--'},{}],
#                                   plot_xmax_value=insetplot_xmax_value,
#                                   plot_xmin_value=insetplot_xmin_value)
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  usplit_data[noise_level][data_idx,::plt_dsample,::plt_dsample,0]],
                                  inset_min_labelsize,
                                  label_list=['Ch1', 'Pred1'],
                                  color_list=[color_ch_list[0], color_pred],
                                #   plot_kwargs_list=[{},{'linestyle':'--'},{}],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

# adding input to the inset.
# sns.kdeplot(data=,
#                     ax=inset_ax,
#                     color=color_ch_list[0],
#                     label='',
#                     clip=(insetplot_xmin_value, None),
#                     )

inset_ax.set_xticks([])
inset_ax.set_yticks([])
add_psnr_str(ax_temp, get_psnr_str(usplit_data[noise_level], 0))
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2,c0_extra+grid_img_sz * 2:c0_extra+grid_img_sz * 3])
ax_temp.imshow(usplit_data[noise_level][data_idx,...,1], cmap='magma')
# inset_ax = add_pixel_kde(ax_temp,
#                                   inset_rect,
#                                   [highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
#                                    noisy_data[data_idx,::plt_dsample,::plt_dsample,1],
#                                   usplit_data[noise_level][data_idx,::plt_dsample,::plt_dsample,1]],
#                                   inset_min_labelsize,
#                                   label_list=['Ch2','input','Pred2'],
#                                   color_list=[color_ch_list[1],color_ch_list[1], color_pred],
#                                   plot_kwargs_list=[{},{'linestyle':'--'},{}],
#                                   plot_xmax_value=insetplot_xmax_value,
#                                   plot_xmin_value=insetplot_xmin_value)
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
                                #    noisy_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  usplit_data[noise_level][data_idx,::plt_dsample,::plt_dsample,1]],
                                  inset_min_labelsize,
                                  label_list=['Ch2','Pred2'],
                                  color_list=[color_ch_list[1],color_ch_list[1], color_pred],
                                #   plot_kwargs_list=[{},{'linestyle':'--'},{}],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
add_psnr_str(ax_temp, get_psnr_str(usplit_data[noise_level], 1))
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[:grid_img_sz, c0_extra+grid_img_sz * 3:c0_extra+grid_img_sz * 4])
ax_temp.imshow(denoiSplitNM_data[noise_level][data_idx,...,0], cmap='magma')
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  denoiSplitNM_data[noise_level][data_idx,::plt_dsample,::plt_dsample,0]],
                                  inset_min_labelsize,
                                  label_list=['Ch1','Pred1'],
                                  color_list=[color_ch_list[0],color_pred],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
inset_ax.set_xticks([])
inset_ax.set_yticks([])

add_psnr_str(ax_temp, get_psnr_str(denoiSplitNM_data[noise_level], 0))
clean_ax(ax_temp)
ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2, c0_extra+grid_img_sz * 3:c0_extra+grid_img_sz * 4])
ax_temp.imshow(denoiSplitNM_data[noise_level][data_idx,...,1], cmap='magma')
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  denoiSplitNM_data[noise_level][data_idx,::plt_dsample,::plt_dsample,1]],
                                  inset_min_labelsize,
                                  label_list=['Ch2','Pred2'],
                                  color_list=[color_ch_list[1],color_pred],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
inset_ax.set_xticks([])
inset_ax.set_yticks([])

add_psnr_str(ax_temp, get_psnr_str(denoiSplitNM_data[noise_level], 1))
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[:grid_img_sz, 2*c0_extra+grid_img_sz * 4:2*c0_extra+grid_img_sz * 5])
ax_temp.imshow(highsnr_data[data_idx,...,0], cmap='magma')
legend_ch1_ax = ax_temp
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [highsnr_data[data_idx,::plt_dsample,::plt_dsample,0]],
                                  inset_min_labelsize,
                                  label_list=['Ch1'],
                                  color_list=[color_ch_list[0]],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

inset_ax.set_xticks([])
inset_ax.set_yticks([])

clean_ax(ax_temp)


ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2, 2*c0_extra+grid_img_sz * 4:2*c0_extra+grid_img_sz * 5])
ax_temp.imshow(highsnr_data[data_idx,...,1], cmap='magma')
inset_ax = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  [highsnr_data[data_idx,::plt_dsample,::plt_dsample,1]],
                                  inset_min_labelsize,
                                  label_list=['Ch2'],
                                  color_list=[color_ch_list[1]],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
legend_ch2_ax = ax_temp

inset_ax.set_xticks([])
inset_ax.set_yticks([])

clean_ax(ax_temp)

# add noise models. 
nmodel1, nmodel2 = get_noise_models(denoiSplitNM_fname[noise_level])

ax_temp = fig.add_subplot(gs[grid_img_sz+1:int(grid_img_sz * 3/2) -1, 2:grid_img_sz])
# ax_temp = fig.add_subplot(gs[grid_img_sz+grid_img_sz//4:grid_img_sz//4 + int(grid_img_sz * 3/2)+1, 1:1+grid_img_sz//2])
clean_ax(ax_temp)
plot_noise_model(40, 90, nmodel1['hist'], nmodel1['gmm'], 'cpu', ax_temp, linetxt_offset=0.2)

ax_temp = fig.add_subplot(gs[int(grid_img_sz * 3/2)+2:2*grid_img_sz -1, 2:grid_img_sz])
# ax_temp = fig.add_subplot(gs[grid_img_sz + grid_img_sz//4:grid_img_sz//4 + int(grid_img_sz * 3/2)+1, grid_img_sz//2+1:grid_img_sz])
clean_ax(ax_temp)
plot_noise_model(40, 90, nmodel2['hist'], nmodel2['gmm'], 'cpu', ax_temp, linetxt_offset=0.2)

# ax_temp = fig.add_subplot(gs[grid_img_sz:int(grid_img_sz * 3/2), :grid_img_sz])
# plot_noise_model(45, 100, nmodel1['hist'], nmodel1['gmm'], 'cpu', ax_temp)

# manually setting legends
import matplotlib.lines as mlines
line_ch1 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[0], linestyle='-', label='$C_1$')
line_ch2 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[1], linestyle='-', label='$C_2$')
line_pred = mlines.Line2D([0, 1], [0, 1], color=color_pred, linestyle='-', label='Pred')
line_noisych1 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[0], linestyle='--', label='$C^N_1$')
line_noisych2 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[1], linestyle='--', label='$C^N_2$')

legend_ch1 = legend_ch1_ax.legend(handles=[line_ch1, line_noisych1, line_pred], loc='upper right', frameon=False, labelcolor='white', 
                          prop={'size': 11})
legend_ch2 = legend_ch2_ax.legend(handles=[line_ch2, line_noisych2, line_pred], loc='upper right', frameon=False, labelcolor='white',
                            prop={'size': 11})
# legend = legend_ax.legend(handles=[line_ch1, line_noisych1, line_ch2, line_noisych2, line_pred], loc='upper left', frameon=False, labelcolor='white', 
#                           prop={'size': 11})

fpath = get_output_fpath(noise_level)
# plt.savefig(fpath, dpi=200, bbox_inches='tight')
print(f'Saved to {fpath}')

In [ ]:
plt.imshow(usplit_data[noise_levels[0]][0,:500,:500, 0])